In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import itertools

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
df = pd.read_csv("./churn_data.csv")
df.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [3]:
df['Exited'].value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df, df['Exited'], random_state=0)

In [5]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    

class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
    
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

Зададим списки признаков

In [6]:
categorical_columns = ['Geography', 'Gender', 'Tenure', 'HasCrCard', 'IsActiveMember']
continuous_columns = ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

Теперь нам нужно под каждый признак создать трансформер и объединить их в список.

In [9]:
from sklearn.preprocessing import StandardScaler

final_transformers = list()

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in continuous_columns:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('scaler', StandardScaler())
            ])
    
    final_transformers.append((cont_col, cont_transformer))


feats = FeatureUnion(final_transformers)

### 1. Для нашего пайплайна (Case1) поэкспериментировать с разными моделями:
    - бустинг
    - логистическая регрессия

##### 1. LogisticRegression()

In [10]:
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

In [11]:
model_lr = Pipeline([
    ('features', feats),
    ('classifier', LogisticRegression(random_state=42)),
])


# обучим пайплайн на всем тренировочном датасете
model_lr.fit(X_train, y_train)

preds = model_lr.predict_proba(X_test)[:, 1]

precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')                                                               

Best Threshold=0.28952195521683477, F-Score=0.510, Precision=0.462, Recall=0.568


In [12]:
metrics_df = pd.DataFrame(columns=['model', 'thresh', 'F-Score', 'Precision', 'Recall', 'ROC AUC'])
metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC


In [13]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.7720774921330664

In [14]:
df = {
    'model': type(model_lr['classifier']).__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc}
df = pd.DataFrame.from_dict(df, orient='index').T
metrics_df = pd.concat([metrics_df, df], axis = 0).reset_index(drop=True)

metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC
0,LogisticRegression,0.289522,0.5097,0.4624,0.56778,0.772077


##### 2. GradientBoostingClassifier()

In [15]:
from sklearn.ensemble import GradientBoostingClassifier


model_gb = Pipeline([
    ('features', feats),
    ('classifier', GradientBoostingClassifier(random_state=42)),
])


# обучим пайплайн на всем тренировочном датасете
model_gb.fit(X_train, y_train)

preds = model_gb.predict_proba(X_test)[:, 1]

precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')    

Best Threshold=0.4085078904556646, F-Score=0.646, Precision=0.704, Recall=0.597


In [16]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.8757458662211781

In [17]:
df = {
    'model': type(model_lr['classifier']).__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc}
df = pd.DataFrame.from_dict(df, orient='index').T
metrics_df = pd.concat([metrics_df, df], axis = 0).reset_index(drop=True)

metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC
0,LogisticRegression,0.289522,0.5097,0.4624,0.56778,0.772077
1,LogisticRegression,0.408508,0.646121,0.703704,0.59725,0.875746


### 2. Отобрать лучшую модель по метрикам (какая по вашему мнению здесь наиболее подходящая ML-метрика)

In [18]:
metrics_df.sort_values('F-Score')

,model,thresh,F-Score,Precision,Recall,ROC AUC
0,LogisticRegression,0.289522,0.5097,0.4624,0.56778,0.772077
1,LogisticRegression,0.408508,0.646121,0.703704,0.59725,0.875746


### 3. Для отобранной модели (на отложенной выборке) сделать оценку экономической эффективности при тех же вводных, как в вопросе 2:
    - 1 доллар на удержание
    - 2 доллара - с каждого правильно классифицированного (True Positive)

In [19]:
preds = model_gb.predict_proba(X_test)[:, 1]

precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)

cnf_matrix = confusion_matrix(y_test, preds > thresholds[ix])

In [20]:
TN = cnf_matrix[0][0]
FP = cnf_matrix[0][1]
FN = cnf_matrix[1][0]
TP = cnf_matrix[1][1]


retain_sum = (FP + TP) * 1
income = TP * 2

income - retain_sum

175

### 4. *Провести подбор гиперпараметров лучшей модели по итогам 2-3

In [21]:
from sklearn.model_selection import GridSearchCV

params = {
    'classifier__max_features': [0.3, 0.5, 0.7],
    'classifier__min_samples_leaf': [1, 15, 30, 50],
    'classifier__n_estimators': [50, 100, 150, 300]
}

In [22]:
%%time
grid = GridSearchCV(model_gb,
                    param_grid=params,
                    cv=5,
                    scoring='recall',
                    refit=False)

search = grid.fit(X_train, y_train)
search.best_params_

CPU times: total: 4min 6s
Wall time: 4min 7s


{'classifier__max_features': 0.7,
 'classifier__min_samples_leaf': 30,
 'classifier__n_estimators': 300}

In [23]:
search.best_params_

{'classifier__max_features': 0.7,
 'classifier__min_samples_leaf': 30,
 'classifier__n_estimators': 300}

In [24]:
from sklearn.ensemble import GradientBoostingClassifier


model_gb_tuned = Pipeline([
    ('features', feats),
    ('classifier', GradientBoostingClassifier(n_estimators=300,
                                              min_samples_leaf=1,
                                              max_features=0.7,
                                              random_state=42)),
])


# обучим пайплайн на всем тренировочном датасете
model_gb_tuned.fit(X_train, y_train)

preds = model_gb_tuned.predict_proba(X_test)[:, 1]

precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')                                                     

Best Threshold=0.3642250281425497, F-Score=0.644, Precision=0.648, Recall=0.640


5. *Еще раз провести оценку экономической эффективности

In [25]:
preds = model_gb_tuned.predict_proba(X_test)[:, 1]

precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)

cnf_matrix = confusion_matrix(y_test, preds > thresholds[ix])

In [26]:
TN = cnf_matrix[0][0]
FP = cnf_matrix[0][1]
FN = cnf_matrix[1][0]
TP = cnf_matrix[1][1]


retain_sum = (FP + TP) * 1
income = TP * 2

income - retain_sum

148